In [ ]:
from dnn_split import *

# already ran: '1M-loads-split.csv', 'compute_fp_1-loads-split.csv'

# files = ['compute_fp_4-loads-split.csv', 'compute_fp_9-loads-split.csv', 'compute_fp_10-loads-split.csv', \
#          'compute_int_0-loads-split.csv', 'compute_int_4-loads-split.csv', 'compute_int_7-loads-split.csv', 'compute_int_11-loads-split.csv', 'compute_int_15-loads-split.csv',\
#          'srv_10-loads-split.csv']

files = ['1M-loads-split.csv']

results = pd.DataFrame(columns = ['Error'])

for file in files:
    print(file)
    dflo, dfhi, y0_range, y1_range = prepDataFrame(file)

    num_training_examples = int(0.8*len(dflo))
    Xlo_tr, Xlo_te, ylo_tr, ylo_te = test_train_split(dflo, num_training_examples)
    Xhi_tr, Xhi_te, yhi_tr, yhi_te = test_train_split(dfhi, num_training_examples)

    # Build the model for predicting LOW bits
    model_lo = build_model(Xlo_tr.shape, 50, 75, 0.001)
    # Train the neural network
    model_lo.fit(Xlo_tr, ylo_tr, epochs=5) # Verbose is not 0 so we can see progress
    # Build the model for predicting HIGH bits
    model_hi = build_model(Xhi_tr.shape, 50, 75, 0.001)
    # Train the neural network
    model_hi.fit(Xhi_tr, yhi_tr, epochs=5) # Verbose is not 0 so we can see progress

    # Get predictions for the test set for LOW bits
    predictions_lo = model_lo.predict(Xlo_te)
    # Get predictions for the test set for HIGH bits
    predictions_hi = model_lo.predict(Xhi_te)

    pdf = pd.DataFrame(predictions_lo, columns=['pred_lo'])
    y_range = y1_range*2**32 + y0_range
    pdf['pred_hi'] = pd.Series(predictions_hi[:,0])
    pdf['truth'] = (pd.Series(yhi_te)*y1_range*2**32 + pd.Series(ylo_te)*y0_range)/y_range
    pdf['pred']  = (pdf['pred_hi']*y1_range*2**32 + pdf['pred_lo']*y0_range)/y_range

    # Compute the root mean square error
    error = sqrt(((pdf['truth'] - pdf['pred'])**2).sum())/len(pdf)
    results.loc[file] = [error]

In [ ]:
results

In [ ]:
results.abs().to_csv('dnn-split-RESULTS-theRest.csv')